In [1]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os

attributes = ['Timestamp', 'canID', 'DLC', 
                           'Data0', 'Data1', 'Data2', 
                           'Data3', 'Data4', 'Data5', 
                           'Data6', 'Data7', 'Flag']

2023-08-10 18:24:32.409950: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 18:24:32.458544: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 18:24:33.156377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import t

# Load model and process test data

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

print(sys.path)

['/home/tiendat/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/torchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [3]:
import torch
import torch.nn as nn
import math
from dataPreprocess import DatasetPreprocess
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import os
import time
import warnings
import torch.nn.functional as F
import time

In [4]:
def draw_confusion(label_y, pre_y, path):
    confusion = confusion_matrix(label_y, pre_y)
    print(confusion)

def write_result(label_y, pre_y, classes_num):
    if classes_num > 2:
        accuracy = accuracy_score(label_y, pre_y)
        macro_precision = precision_score(label_y, pre_y, average='macro')
        macro_recall = recall_score(label_y, pre_y, average='macro')
        macro_f1 = f1_score(label_y, pre_y, average='macro')
        micro_precision = precision_score(label_y, pre_y, average='micro')
        micro_recall = recall_score(label_y, pre_y, average='micro')
        micro_f1 = f1_score(label_y, pre_y, average='micro')
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- macro precision: ', macro_precision)
        print('    -- macro recall: ', macro_recall)
        print('    -- macro f1 score: ', macro_f1)
        print('    -- micro precision: ', micro_precision)
        print('    -- micro recall: ', micro_recall)
        print('    -- micro f1 score: ', micro_f1)
        report = classification_report(label_y, pre_y)
        print(report)
    else:
        accuracy = accuracy_score(label_y, pre_y)
        precision = precision_score(label_y, pre_y)
        recall = recall_score(label_y, pre_y)
        f1 = f1_score(label_y, pre_y)
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- recall: ', recall)
        print('    -- precision: ', precision)
        print('    -- f1 score: ', f1)
        report = classification_report(label_y, pre_y)
        print(report)

In [5]:
class Config:
    def __init__(self):
        self.model_name = 'Transformer'
        self.slide_window = 1
        self.slsum_count = 8 #int(math.pow(16, self.slide_window))  # 滑动窗口计数的特征的长度 n-gram?
        self.dnn_out_d = 8 # 经过DNN后的滑动窗口计数特征的维度 Dimensions of sliding window count features after DNN 8
        self.head_dnn_out_d = 32 
        self.d_model = self.dnn_out_d + self.head_dnn_out_d # transformer的输入的特征的维度, dnn_out_d + 包头长度 The dimension of the input feature of the transformer, dnn_out_d + header length
        self.pad_size = 29
        self.window_size = 29
        self.max_time_position = 10000
        self.nhead = 5 # ori: 5
        self.num_layers = 5
        self.gran = 1e-6
        self.log_e = 2
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.classes_num = 5 
        self.batch_size = 10
        self.epoch_num = 20
        self.lr = 0.001 #0.00001 learning rate 
        self.train_pro = 0.7  # 训练集比例 Ratio of training set

        self.root_dir = '../road/twomode/TFRecord_w29_s29/1/'
        self.model_save_path = '../model/' + self.model_name + '/'
        if not os.path.exists(self.model_save_path):
            os.mkdir(self.model_save_path)
        self.result_file = '/mnt/hdd2/transformer-entropy-ids/result/trans8_performance.txt'

        self.isload_model = False  # 是否加载模型继续训练 Whether to load the model and continue training
        self.start_epoch = 18  # 加载的模型的epoch The epoch of the loaded model
        # self.model_path = '../model/' + self.model_name + '/' + self.model_name + '_model_' + str(self.start_epoch) + '.pth'  # 要使用的模型的路径 path to the model to use
        self.model_path = '../model/Transformer/best_model.pth'
        self.model_path = '../model/Transformer/Transformer_model_141.pth'
        
class DNN(nn.Module):
    def __init__(self, d_in, d_out):  # config.slsum_count, config.dnn_out_d
        super(DNN, self).__init__()
        self.l1 = nn.Linear(d_in, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, d_out)

    def forward(self, x):
        # print('x: ', x.numpy()[0])
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        # print('dnn out: ', out.detach().numpy()[0])
        return out

class Time_Positional_Encoding(nn.Module):
    def __init__(self, embed, max_time_position, device):
        super(Time_Positional_Encoding, self).__init__()
        self.device = device

    def forward(self, x, time_position):
        out = x.permute(1, 0, 2)
        out = out + nn.Parameter(time_position, requires_grad=False).to(self.device)
        out = out.permute(1, 0, 2)
        return out

class MyTrans(nn.Module):
    def __init__(self, config):
        super(MyTrans, self).__init__()
        self.dnn = DNN(config.slsum_count, config.dnn_out_d).to(config.device)
        self.head_dnn = DNN(4, config.head_dnn_out_d).to(config.device)
        self.position_embedding = Time_Positional_Encoding(config.d_model, config.max_time_position, config.device).to(config.device)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, nhead=config.nhead).to(config.device)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=config.num_layers).to(config.device)
        self.fc = nn.Linear(config.d_model, config.classes_num).to(config.device)
        self.pad_size = config.pad_size
        self.dnn_out_d = config.dnn_out_d
        self.head_dnn_out_d = config.head_dnn_out_d

    def forward(self, header, sl_sum, mask, time_position):
        dnn_out = torch.empty((sl_sum.shape[0], self.dnn_out_d, 0)).to(config.device)

        for i in range(self.pad_size):
            tmp = self.dnn(sl_sum[:, i, :]).unsqueeze(2)
            dnn_out = torch.concat((dnn_out, tmp), dim=2)
        dnn_out = dnn_out.permute(0, 2, 1)

        head_dnn_out = torch.empty((header.shape[0], self.head_dnn_out_d, 0)).to(config.device)
        for i in range(self.pad_size):
            tmp = self.head_dnn(header[:, i, :]).unsqueeze(2)
            head_dnn_out = torch.concat((head_dnn_out, tmp), dim=2)
        head_dnn_out = head_dnn_out.permute(0, 2, 1)

        x = torch.concat((head_dnn_out, dnn_out), dim=2).permute(1, 0, 2)

        out = self.position_embedding(x, time_position)
        out = self.transformer_encoder(out, src_key_padding_mask=mask)
        out = out.permute(1, 0, 2)
        out = torch.sum(out, 1)
        out = self.fc(out)
        return out


In [6]:
config = Config()
print(config.device)
print(config.model_path)

seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

train_dataset = DatasetPreprocess(config.root_dir, config.window_size, config.pad_size, config.d_model, config.max_time_position, config.gran, config.log_e, is_train=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)

test_dataset = DatasetPreprocess(config.root_dir, config.window_size, config.pad_size, config.d_model, config.max_time_position, config.gran, config.log_e, is_train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)
print('finish load data')

model = torch.load(config.model_path)
print("Model loaded")

loss_func = nn.CrossEntropyLoss().to(config.device)
opt = torch.optim.Adam(model.parameters(), lr=config.lr)

cuda
../model/Transformer/Transformer_model_141.pth
finish load data
Model loaded


In [10]:
linked_dict = {}

with torch.no_grad():
    for j, train_batch in enumerate(train_loader):
        # print(f"Origin time: {train_batch['oritime']} and length: {len(train_batch['oritime'])}")
        # aa
        canid = []
        message = []
        time_stamp = [] 
        
        for index in range(len(train_batch['oritime'])): 
            time_stamp.extend(train_batch['oritime'][index].tolist())
        
            for msg in train_batch['header'][index]:
                # print(f"{msg}")
                canid.append(''.join(map(str, msg.numpy())))
                # print(f"{''.join(map(str, msg))}")
            
            for msg in train_batch['payload'][index]:
                message.append(msg.numpy().tolist())
                # print(f"{''.join(map(str, msg))}")
        # print(canid)
        # print(len(message))
        # print(len(time_stamp))
        
        # Populate the dictionary
        for i, id in enumerate(canid):
            if id in linked_dict:
                linked_dict[id]['payload'].append(message[i])
                linked_dict[id]['timestamp'].append(time_stamp[i])
            else:
                linked_dict[id] = {'payload': [message[i]], 'timestamp': [time_stamp[i]]}
                
for can_id in linked_dict:
    sorted_indices = sorted(range(len(linked_dict[can_id]['timestamp'])), key=lambda k: linked_dict[can_id]['timestamp'][k])
    linked_dict[can_id]['timestamp'] = [linked_dict[can_id]['timestamp'][i] for i in sorted_indices]
    linked_dict[can_id]['payload'] = [linked_dict[can_id]['payload'][i] for i in sorted_indices]


In [11]:
count = 0
for can_id in linked_dict:
    count+=1
    time_diffs = [linked_dict[can_id]['timestamp'][i+1] - linked_dict[can_id]['timestamp'][i] for i in range(len(linked_dict[can_id]['timestamp']) - 1)]
    max_time_diff = max(time_diffs)
    min_time_diff = min(time_diffs)
    linked_dict[can_id]['max_time'] = max_time_diff
    linked_dict[can_id]['min_time'] = min_time_diff
    # print(f"CANID: {can_id} with max: {linked_dict[can_id]['max_time']} and min: {linked_dict[can_id]['min_time']}")


In [12]:
for can_id in linked_dict:
    payload_unique = []
    for payload in linked_dict[can_id]['payload']:
        if payload not in payload_unique:
            payload_unique.append(payload)    
    linked_dict[can_id]['uni_payload'] = payload_unique
    print(f"CANID: {can_id} ori: {len(linked_dict[can_id]['payload'])} with {len(linked_dict[can_id]['uni_payload'])}")

CANID: 0295 ori: 11800 with 5
CANID: 0107 ori: 17671 with 2
CANID: 00130 ori: 47313 with 30915
CANID: 0033 ori: 35308 with 7941
CANID: 03213 ori: 7056 with 4
CANID: 041513 ori: 7065 with 1639
CANID: 02141 ori: 17639 with 2
CANID: 0354 ori: 35277 with 3897
CANID: 0193 ori: 17659 with 68
CANID: 05141 ori: 35270 with 4607
CANID: 02014 ori: 17661 with 8908
CANID: 0522 ori: 17635 with 2166
CANID: 0125 ori: 35251 with 1572
CANID: 0162 ori: 17599 with 1224
CANID: 0366 ori: 17679 with 58
CANID: 0580 ori: 17636 with 3300
CANID: 0434 ori: 17623 with 15055
CANID: 06140 ori: 39117 with 23805
CANID: 00107 ori: 35316 with 3833
CANID: 0498 ori: 17596 with 10620
CANID: 0230 ori: 17563 with 2190
CANID: 02121 ori: 3362 with 3200
CANID: 00014 ori: 35272 with 1776
CANID: 0153 ori: 17654 with 8
CANID: 0662 ori: 17655 with 35
CANID: 01912 ori: 17670 with 5487
CANID: 0274 ori: 17658 with 2929
CANID: 00313 ori: 3556 with 496
CANID: 00137 ori: 3470 with 2
CANID: 05512 ori: 2370 with 7
CANID: 041211 ori: 1757 w

In [17]:
[linked_dict['02811']['payload'][i] for i in range(20)]
print(len(linked_dict['0230']['payload']))
# for i in linked_dict['0230']['payload']:
#     print(i.tolist())
#     aa

17563


In [14]:
import time

def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    max_distance = 0
    min_distance = 64
    print(f"")
    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            # Chỉ so sánh với 00000000000000 rồi trừ ra
            # print(f"{max_distance} and {min_distance}")
            # aa
            
    return max_distance, min_distance

for can_id in linked_dict:
    print(f"\n{can_id}:{len(linked_dict[can_id]['uni_payload'])} processing...")
    start_time = time.time()
    if len(linked_dict[can_id]['uni_payload']) > 1:
        max_distance, min_distance = all_pair_hamming_distance_int_list(linked_dict[can_id]['uni_payload'])
        # print(f"CANID: {can_id} with max: {linked_dict[can_id]['max_time']} and min: {linked_dict[can_id]['min_time']}")
        linked_dict[can_id]['max_hmc'] = max_distance
        linked_dict[can_id]['min_hmc'] = min_distance
    else:
        linked_dict[can_id]['max_hmc'] = 0
        linked_dict[can_id]['min_hmc'] = 0
    # for pair, distance in pair_distances.items():
    #     idx1, idx2 = pair
    #     print(f"Hamming distance between {linked_dict[can_id]['payload'][idx1]} and {linked_dict[can_id]['payload'][idx2]}: {distance}")
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"CANID: {can_id} max distance: {max_distance} and min distance: {min_distance} on {execution_time} seconds ==")



0295:5 processing...

CANID: 0295 max distance: 3 and min distance: 1 on 7.43865966796875e-05 seconds ==

0107:2 processing...

CANID: 0107 max distance: 64 and min distance: 64 on 1.1205673217773438e-05 seconds ==

00130:30915 processing...

CANID: 00130 max distance: 52 and min distance: 1 on 1444.4867849349976 seconds ==

0033:7941 processing...

CANID: 0033 max distance: 43 and min distance: 1 on 96.84521532058716 seconds ==

03213:4 processing...

CANID: 03213 max distance: 5 and min distance: 1 on 4.100799560546875e-05 seconds ==

041513:1639 processing...

CANID: 041513 max distance: 32 and min distance: 1 on 4.0089335441589355 seconds ==

02141:2 processing...

CANID: 02141 max distance: 63 and min distance: 63 on 1.6450881958007812e-05 seconds ==

0354:3897 processing...

CANID: 0354 max distance: 56 and min distance: 1 on 22.735036373138428 seconds ==

0193:68 processing...

CANID: 0193 max distance: 8 and min distance: 1 on 0.007156848907470703 seconds ==

05141:4607 proces

In [15]:
for can_id in linked_dict:
    print(f"\nCANID {can_id} and lenght unique {len(linked_dict[can_id]['uni_payload'])}:")
    print(f"Max time interval: {linked_dict[can_id]['max_time']} and min time interval: {linked_dict[can_id]['min_time']}")
    print(f"Max hamming {linked_dict[can_id]['max_hmc']} and min hamming {linked_dict[can_id]['min_hmc']}")


CANID 0295 and lenght unique 5:
Max time interval: 0.11974334716796875 and min time interval: 0.0
Max hamming 3 and min hamming 1

CANID 0107 and lenght unique 2:
Max time interval: 0.0800628662109375 and min time interval: 0.0
Max hamming 64 and min hamming 64

CANID 00130 and lenght unique 30915:
Max time interval: 0.059967041015625 and min time interval: 0.0
Max hamming 52 and min hamming 1

CANID 0033 and lenght unique 7941:
Max time interval: 0.059967041015625 and min time interval: 0.0
Max hamming 43 and min hamming 1

CANID 03213 and lenght unique 4:
Max time interval: 0.24977874755859375 and min time interval: 0.0
Max hamming 5 and min hamming 1

CANID 041513 and lenght unique 1639:
Max time interval: 0.15065765380859375 and min time interval: 2.86102294921875e-06
Max hamming 32 and min hamming 1

CANID 02141 and lenght unique 2:
Max time interval: 0.0805816650390625 and min time interval: 0.0
Max hamming 63 and min hamming 63

CANID 0354 and lenght unique 3897:
Max time inter

In [16]:
print((linked_dict['00130']['uni_payload'][0]))
a = [42, 119, 4, 96, 247, 0, 0, 0]
max_distance = 0
min_distance = 64

start_time = time.time()
for payload in linked_dict['00130']['uni_payload']:
    distance = hamming_distance_int_list(a, payload)
    max_distance = max(max_distance, distance)
    min_distance = min(min_distance, distance)
end_time = time.time()
  
print(f"max {max_distance} and {min_distance}")
print(f"time {end_time - start_time} seconds")

[42, 119, 4, 96, 247, 0, 0, 0]
max 45 and 0
time 0.1266934871673584 seconds


In [45]:
with open('../rule_static2.json') as f:
    rule_static = json.load(f)


for can_id in rule_static:
    print(f"\nCANID {can_id} and lenght unique {len(rule_static[can_id]['uni_payload'])}:")
    print(f"Max time interval: {rule_static[can_id]['max_time']} and min time interval: {rule_static[can_id]['min_time']}")
    print(f"Max hamming {rule_static[can_id]['max_hmc']} and min hamming {rule_static[can_id]['min_hmc']}")


CANID 0295 and lenght unique 5:
Max time interval: 0.11974334716796875 and min time interval: 0.0
Max hamming 3 and min hamming 1

CANID 0107 and lenght unique 2:
Max time interval: 0.0800628662109375 and min time interval: 0.0
Max hamming 64 and min hamming 64

CANID 00130 and lenght unique 30915:
Max time interval: 0.059967041015625 and min time interval: 0.0
Max hamming 52 and min hamming 1

CANID 0033 and lenght unique 7941:
Max time interval: 0.059967041015625 and min time interval: 0.0
Max hamming 43 and min hamming 1

CANID 03213 and lenght unique 4:
Max time interval: 0.24977874755859375 and min time interval: 0.0
Max hamming 5 and min hamming 1

CANID 041513 and lenght unique 1639:
Max time interval: 0.15065765380859375 and min time interval: 2.86102294921875e-06
Max hamming 32 and min hamming 1

CANID 02141 and lenght unique 2:
Max time interval: 0.0805816650390625 and min time interval: 0.0
Max hamming 63 and min hamming 63

CANID 0354 and lenght unique 3897:
Max time inter

In [59]:
canid_list = list(rule_static.keys())
print(canid_list)

['0295', '0107', '00130', '0033', '03213', '041513', '02141', '0354', '0193', '05141', '02014', '0522', '0125', '0162', '0366', '0580', '0434', '06140', '00107', '0498', '0230', '02121', '00014', '0153', '0662', '01912', '0274', '00313', '00137', '05512', '041211', '0280', '06914', '00120', '06612', '02811', '0407', '0585', '03121', '0577', '0684', '0636', '02137', '03144', '06512', '001110', '00151', '02142', '0345', '0430', '0130', '05113', '0464', '0497', '011210', '02103', '00154', '0297', '0075', '00310', '00158', '02114', '02132', '04147', '061512', '0239', '00312', '0207', '04129', '01136', '0618', '02614', '0025', '041414', '0277', '05111', '01124', '02104', '06713', '001212', '0576', '0419', '0533', '06913', '05148', '0273', '021011', '06137', '01212', '0371', '051513', '051015', '001513', '01104', '0671', '0041', '02113', '03010', '02117', '03102', '0655', '03119', '01145', '011010', '0006']


In [96]:
if any([0,0,0,0,1]):
    print("hahaha")
7.8/7900

hahaha


0.0009873417721518987

In [145]:
label_y = []
pre_y = []
rule_pre_y = []

def whitelisted_canid_check(temp_canid, whitelisted_canid_list):
    if temp_canid not in whitelisted_canid_list:
        return 1
    return 0

def time_interval_check(index, test_batch, rule_static):
    temp_pre = []
    for jdex in range(len(test_batch['oritime'][index].numpy())):
        temp_canid = ''.join(map(str, test_batch['header'][index][jdex].numpy()))
        print(f"{temp_canid} and {test_batch['oritime'][index][jdex]}")
        aa
    print(time_batch[index][0])
    # canid_batch include 10 message
    return 0

def hamming_distance_check(canid_batch, whitelisted_canid_list):
    a = [42, 119, 4, 96, 247, 0, 0, 0]
    max_distance = 0
    min_distance = 64
    
    # canid_batch include 10 message
    for msg in canid_batch:
        temp_canid = ''.join(map(str, msg.numpy()))
        
    for payload in whitelisted_canid_list['uni_payload']:
        distance = hamming_distance_int_list(a, payload)
        max_distance = max(max_distance, distance)
        min_distance = min(min_distance, distance)
    
    print(f"max {max_distance} and {min_distance}")
    print(f"time {end_time - start_time} seconds")
    return


with torch.no_grad():
    # each test_batch have 10 batch
    for j, test_batch in enumerate(test_loader):
        # index 0->10
        for index in range(len(test_batch['oritime'])): 
            rule_pre_y = []
            # iter 0 -> 29
            
            for can_id in test_batch['header'][index].numpy():
                temp_canid = ''.join(map(str, can_id))
                # print(whitelisted_canid_check(temp_canid, canid_list))
            
            print(time_interval_check(index, test_batch, rule_static))
            aa
    print(f"{len(rule_pre_y)}")
        
            # for msg in test_batch['payload'][index]:
            #     message.append(msg.numpy().tolist())
            #     # print(f"{''.join(map(str, msg))}")

0193 and 23.565410614013672


NameError: name 'aa' is not defined

In [ ]:
with torch.no_grad():
    for j, test_sample_batch in enumerate(test_loader):
        test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
        test_sl_sum = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
        test_mask = test_sample_batch['mask'].to(config.device)
        test_label = test_sample_batch['label'].to(config.device)
        test_time_position = test_sample_batch['time'].to(config.device)

# TEST

In [7]:
label_y = []
pre_y = []
timeCalc = [] 
with torch.no_grad():
    for j, test_sample_batch in enumerate(test_loader):
        start_time = time.time()
        test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
        test_sl_sum = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
        test_mask = test_sample_batch['mask'].to(config.device)
        test_label = test_sample_batch['label'].to(config.device)
        test_time_position = test_sample_batch['time'].to(config.device)
        
        test_out = model(test_header, test_sl_sum, test_mask, test_time_position)

        pre = torch.max(test_out, 1)[1].cpu().numpy() # the predict result
        
        pre_y = np.concatenate([pre_y, pre], 0)
        label_y = np.concatenate([label_y, test_label.cpu().numpy()], 0)
        end_time = time.time()
        execution_time = end_time - start_time
        timeCalc.append(execution_time)
    write_result(label_y, pre_y, config.classes_num)
    draw_confusion(label_y, pre_y, '')
    


  -- test result: 
    -- accuracy:  0.9989947222920331
    -- macro precision:  0.9990287768869496
    -- macro recall:  0.9989483396147361
    -- macro f1 score:  0.9989883010067664
    -- micro precision:  0.9989947222920331
    -- micro recall:  0.9989947222920331
    -- micro f1 score:  0.9989947222920331
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4294
         1.0       1.00      1.00      1.00      3664

    accuracy                           1.00      7958
   macro avg       1.00      1.00      1.00      7958
weighted avg       1.00      1.00      1.00      7958

[[4292    2]
 [   6 3658]]


In [42]:
print(len(label_y))
print(len(pre_y))

7958
7958


In [27]:
def calculate_average(arr):
    if not arr:
        raise ValueError("Input array is empty. Cannot calculate the average.")
    
    total = sum(arr)
    average = total / len(arr)
    return average
print("Execution time:", calculate_average(timeCalc), "seconds")

Execution time: 0.013575118989800688 seconds


In [ ]:
label_y = []
pre_y = []
timeCalc = [] 
with torch.no_grad():
    for j, test_sample_batch in enumerate(test_loader):
        start_time = time.time()
        test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
        test_sl_sum = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
        test_mask = test_sample_batch['mask'].to(config.device)
        test_label = test_sample_batch['label'].to(config.device)
        test_time_position = test_sample_batch['time'].to(config.device)
        
        # print(f"TEST HEADER: {test_sample_batch['header']}")
        # print(len(test_sample_batch['label']))
        # aa
        
        # for session in test_sample_batch['header']:
        #     for msg in session:
        #         print(f"Message: {''.join(map(str, msg.numpy()))}")
        #     aa 
        
        test_out = model(test_header, test_sl_sum, test_mask, test_time_position)

        pre = torch.max(test_out, 1)[1].cpu().numpy()
        pre_y = np.concatenate([pre_y, pre], 0)
        label_y = np.concatenate([label_y, test_label.cpu().numpy()], 0)
        end_time = time.time()
        execution_time = end_time - start_time
        timeCalc.append(execution_time)
    write_result(label_y, pre_y, config.classes_num)
    draw_confusion(label_y, pre_y, '')

  -- test result: 
    -- accuracy:  0.9999883322054465
    -- macro precision:  0.9999962205319601
    -- macro recall:  0.9999719881743399
    -- macro f1 score:  0.999984103808873
    -- micro precision:  0.9999883322054465
    -- micro recall:  0.9999883322054465
    -- micro f1 score:  0.9999883322054465
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    105833
         1.0       1.00      1.00      1.00     11236
         2.0       1.00      1.00      1.00     13346
         3.0       1.00      1.00      1.00     19585
         4.0       1.00      1.00      1.00     21412

    accuracy                           1.00    171412
   macro avg       1.00      1.00      1.00    171412
weighted avg       1.00      1.00      1.00    171412

[[105833      0      0      0      0]
 [     1  11235      0      0      0]
 [     0      0  13346      0      0]
 [     1      0      0  19584      0]
 [     0      0      0      0  21412]]


# Entropy algorithm

In [ ]:
def hex_to_int(hex_value):
    return int(hex_value, base=16)

def hex_string_to_array(hex_string):
    return ''.join(map(str, list(map(hex_to_int, hex_string))))

def complete_field(sample):
    if not isinstance(sample['Flag'], str):
        col = 'Data' + str(sample['DLC'])
        sample['Flag'] = sample[col]
        sample[col] = '00'
    return sample

def max_min(lst):
    """
    Finds the maximum and minimum values in a list.
    Returns a tuple containing the maximum and minimum values.
    """
    if len(lst) == 0:
        return None
    
    max_val = lst[0]
    min_val = lst[0]
    for val in lst:
        if val > max_val:
            max_val = val
        elif val < min_val:
            min_val = val
    
    return (max_val, min_val)

def extract_time_interval(time_list):
    """
    Calculates the differences between adjacent elements in an ascending array.
    Returns a list containing all differences.
    """
    if len(time_list) < 2:
        return None
    
    differences = []
    for i in range(len(time_list)-1):
        diff = time_list[i+1] - time_list[i]
        differences.append(diff)
    
    return max_min(differences)

def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    hamming_distances = {}
    max_distance = 0
    min_distance = 64

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances, max_distance, min_distance


In [ ]:
def hamming_distance_int(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Input lists must have the same length.")
    
    distance = sum(a != b for a, b in zip(list1, list2))
    return distance

# Example usage:
list1_example = [216, 0, 0, 137, 0, 0, 0, 0]
list2_example = [216, 85, 0, 139, 0, 0, 0, 0]
distance_value = hamming_distance_int(list1_example, list2_example)
print("Hamming distance between the integer lists:", distance_value)


Hamming distance between the integer lists: 2


In [ ]:
def hamming_distance_int_list(seq1, seq2):
    # Calculate the Hamming distance between two binary sequences represented as lists
    return sum(bin(x ^ y).count('1') for x, y in zip(seq1, seq2))

def all_pair_hamming_distance_int_list(sequence_list):
    n = len(sequence_list)
    hamming_distances = {}
    max_distance = 0
    min_distance = 64

    for i in range(n):
        for j in range(i + 1, n):
            distance = hamming_distance_int_list(sequence_list[i], sequence_list[j])
            max_distance = max(max_distance, distance)
            min_distance = min(min_distance, distance)
            hamming_distances[(i, j)] = distance
            # Since Hamming distance is symmetric (H(A, B) == H(B, A)),
            # we can also store the distance for (j, i) pair.
            hamming_distances[(j, i)] = distance
    
    return hamming_distances, max_distance, min_distance

# Example usage:
binary_sequence_list_example = [
    [216, 0, 0, 138, 0, 0, 0, 0],
    [216, 85, 0, 139, 0, 0, 0, 0],
    [216, 0, 0, 137, 0, 0, 0, 0],
    [216, 85, 0, 140, 0, 0, 0, 0],
    [216, 0, 0, 138, 0, 0, 0, 0]
]

pair_distances, max_distance, min_distance = all_pair_hamming_distance_int_list(binary_sequence_list_example)

# Print the Hamming distances for all pairs
for pair, distance in pair_distances.items():
    idx1, idx2 = pair
    print(f"Hamming distance between {binary_sequence_list_example[idx1]} and {binary_sequence_list_example[idx2]}: {distance}")
    
print(f"Max distance: {max_distance} and min distance: {min_distance}")


Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 2
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 6
Hamming distance between [216, 85, 0, 140, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 6
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 0, 0, 138, 0, 0, 0, 0] and [216, 0, 0, 138, 0, 0, 0, 0]: 0
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 0, 0, 137, 0, 0, 0, 0]: 5
Hamming distance between [216, 0, 0, 137, 0, 0, 0, 0] and [216, 85, 0, 139, 0, 0, 0, 0]: 5
Hamming distance between [216, 85, 0, 139, 0, 0, 0, 0] and [216, 85, 0, 140, 0, 0, 0, 0]: 3
Ha

In [ ]:
[216, 000, 000, 138, 000, 062, 000, 000] Normal
[216, 000, 000, 138, 000, 000, 000, 000]  Normal lệch 6
[216, 000, 000, 138, 000, 170, 000, 000]  Attack

Case 1: Max 1 
Case 2: Max 6
Case 3: Max 3

Dùng 2 hamming distance check

In [150]:
bin(170 ^ 62)

'0b10010100'

In [ ]:
import numpy as np

def conditional_entropy(y, x):
    # Convert input to numpy arrays if they are not already
    y = np.array(y)
    x = np.array(x)
    
    # Get unique values and their counts for X and Y
    unique_x, counts_x = np.unique(x, return_counts=True)
    unique_y, counts_y = np.unique(y, return_counts=True)
    
    # Initialize conditional entropy
    h_y_given_x = 0.0
    
    # Calculate conditional entropy
    for xi, cx in zip(unique_x, counts_x):
        # Get the subset of Y corresponding to xi in X
        y_given_xi = y[x == xi]
        
        # Get unique values and their counts for the subset of Y given xi in X
        unique_y_given_xi, counts_y_given_xi = np.unique(y_given_xi, return_counts=True)
        
        # Calculate conditional probabilities P(Y|X) for xi
        p_y_given_xi = counts_y_given_xi / cx
        
        # Calculate conditional entropy contribution for xi
        h_y_given_xi = np.sum(-p_y_given_xi * np.log2(p_y_given_xi))
        
        # Weighted sum based on P(X=xi)
        h_y_given_x += h_y_given_xi * (cx / len(x))
    
    return h_y_given_x

# Example usage:
# Assuming X and Y are two discrete random variables in the form of lists or arrays
X = [1, 1, 1, 0, 1, 1, 0, 0]
Y = [1, 0, 0, 1, 0, 1, 0, 0]

# Calculate H(Y|X)
conditional_entropy_value = conditional_entropy(Y, X)
print("Conditional Entropy H(Y|X) =", conditional_entropy_value)


Conditional Entropy H(Y|X) = 0.9512050593046015


In [ ]:
0.01722097396850586-0.017220020294189453

9.5367431640625e-07

In [10]:
# Given list
lst = [0, 16, 252, 100, 193, 40, 32, 160]

# Convert the list to a single string with each element padded to 3 digits
result = ''.join(f'{item:03}' for item in lst)
result

'000016250100193040030160'

In [31]:
def string_hamming_distance(str1, str2):
    # Ensure both strings are of the same length. If not, pad the shorter one with zeros.
    string1 = ''.join(f'{item:03}' for item in str1)
    string2 = ''.join(f'{item:03}' for item in str2)
    print(string1, string2)

    # Calculate the Hamming distance
    distance = 0
    differing_indices = []
    for i, (ch1, ch2) in enumerate(zip(string1, string2)):
        if ch1 != ch2:
            distance += 1
            differing_indices.append(i)
    
    # Extract start and end index of differing region
    if differing_indices:
        start_index = differing_indices[0]
        end_index = differing_indices[-1]
    else:
        start_index, end_index = 0, 0
    
    return distance, int(start_index/3), int(end_index/3)

string_hamming_distance([0, 16, 252, 100, 193, 40, 32, 160],[0, 16, 252, 100, 193, 40, 32, 160])

000016252100193040032160 000016252100193040032160


(0, 0, 0)

In [15]:
def string_hamming_distance(str1, str2):
    # Ensure both strings are of the same length. If not, pad the shorter one with zeros.
    max_len = max(len(str1), len(str2))
    str1 = str1.rjust(max_len, '0')
    str2 = str2.rjust(max_len, '0')

    # Calculate the Hamming distance
    distance = 0
    differing_indices = []
    for i, (ch1, ch2) in enumerate(zip(str1, str2)):
        if ch1 != ch2:
            distance += 1
            differing_indices.append(i)
    
    # Extract start and end index of differing region
    if differing_indices:
        start_index = differing_indices[0]
        end_index = differing_indices[-1]
    else:
        start_index, end_index = None, None
    
    return distance, int(start_index/3), int(end_index/3)

# Testing the function with the provided example
string_hamming_distance("000016252100193040032160", "000046120100193040030161")


(6, 1, 7)

In [9]:
'595945450000FFFF', 'XXXXXXXXXXFFXXXX', 'XXXX04XXXXXXXXXX', 'XXXX0CXXXXXXXXXX'

if 

0.06381583213806152

In [21]:
'XXXX0CXXXXXXXXXX'.find('0C')

4

In [22]:
a = 'XXXX0CXXXXXXXXXX'
a[4:6]

'0C'

In [1]:
import json

candict = json.load(open('/home/tiendat/transformer-entropy-ids/road/candict.txt', 'r'))

In [ ]:
# print(candict.keys())

for canid in candict:
    print(len(candict[canid]))
    

In [10]:
for canid in candict:
    print(f"CANID {canid} with length unipayload: {len(candict[canid]['uni_payload'])}")
    print(f"Max time {candict[canid]['max_time']} and min time {candict[canid]['max_time']}")


CANID 167 with length unipayload: 119265
Max time 178.6683760881424 and min time 178.6683760881424
CANID 852 with length unipayload: 24923
Max time 178.6693559885025 and min time 178.6693559885025
CANID 1505 with length unipayload: 62906
Max time 178.6693550348282 and min time 178.6693550348282
CANID 208 with length unipayload: 338060
Max time 179.78900504112244 and min time 179.78900504112244
CANID 51 with length unipayload: 125439
Max time 179.78900492191315 and min time 179.78900492191315
CANID 14 with length unipayload: 57632
Max time 173.05416202545166 and min time 173.05416202545166
CANID 293 with length unipayload: 17875
Max time 254.6841459274292 and min time 254.6841459274292
CANID 1760 with length unipayload: 362648
Max time 26.87276494503021 and min time 26.87276494503021
CANID 339 with length unipayload: 79
Max time 179.7957010269165 and min time 179.7957010269165
CANID 263 with length unipayload: 10
Max time 19.76394093036652 and min time 19.76394093036652
CANID 1634 with 

In [41]:
from collections import defaultdict
import numpy as np
import time

def calculate_entropy(X):
    p_x = defaultdict(float)
    n = len(X)
    
    for x in X:
        p_x[x] += 1 / n
    
    H_X = 0
    for x in p_x:
        H_X += -p_x[x] * np.log2(p_x[x])
    
    return H_X

def calculate_conditional_entropy(X, Y):
    p_x = defaultdict(float)
    p_y_given_x = defaultdict(lambda: defaultdict(float))
    
    n = len(X)
    
    for x, y in zip(X, Y):
        p_x[x] += 1 / n
        p_y_given_x[x][y] += 1
    
    for x in p_y_given_x:
        total = sum(p_y_given_x[x].values())
        for y in p_y_given_x[x]:
            p_y_given_x[x][y] /= total
    
    H_Y_given_X = 0
    for x in p_x:
        for y in p_y_given_x[x]:
            H_Y_given_X += p_x[x] * p_y_given_x[x][y] * -np.log2(p_y_given_x[x][y])
    
    return H_Y_given_X

def calculate_mutual_information(X, Y):
    H_X = calculate_entropy(X)
    H_Y = calculate_entropy(Y)
    print(f"H(Y) = {H_Y}")
    H_Y_given_X = calculate_conditional_entropy(X, Y)
    
    I_X_Y = H_Y - H_Y_given_X
    return I_X_Y

# Tập dữ liệu
X = "000000XX03XX0000"
Y = "0000001203FF0000"

# Tính H(Y|X)
start = time.time()
H_Y_given_X = calculate_conditional_entropy(X, Y)
I_X_Y = calculate_mutual_information(X, Y)
print(f"I(X;Y) = {I_X_Y}")
print(f"H(Y|X) = {H_Y_given_X} in {time.time() - start} seconds")


H(Y) = 1.496640762186858
I(X;Y) = 1.121640762186858
H(Y|X) = 0.375 in 0.0005106925964355469 seconds


In [1]:
import numpy as np
import time

# Generate a sample data array of 600,000 strings, each with 24 characters
np.random.seed(42)
X = np.random.choice(list('0123456789'), size=(600000, 24), replace=True)
X = np.array([''.join(row) for row in X.astype(str)])

# Generate a sample target string 'y' with 24 characters
y = ''.join(np.random.choice(list('0123456789'), size=24, replace=True))

# Function to compute Hamming distance using vectorization
def vectorized_hamming_distance(y, X):
    return np.sum(X != y[:, None], axis=1)

# Measure the time taken to compute the Hamming distances
start_time = time.time()

# Compute the Hamming distances
y_array = np.array(list(y))
distances = vectorized_hamming_distance(y_array, X)

# Find the maximum and minimum Hamming distance
max_distance = np.max(distances)
min_distance = np.min(distances)

elapsed_time = time.time() - start_time

max_distance, min_distance, elapsed_time


(600000, 600000, 0.10051393508911133)

In [10]:
from collections import Counter
import time

string = '016064096255120195008000'

start_time = time.time()
frequency = Counter(string)
print(f"Frequency: {frequency} in {time.time() - start_time} seconds")

Frequency: Counter({'0': 9, '1': 3, '6': 3, '5': 3, '9': 2, '2': 2, '4': 1, '8': 1}) in 0.00029468536376953125 seconds


In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# Define the input tensor
x = torch.tensor([[0, 8, 1, 243, 234, 17, 244, 206]]).to(torch.float32)

# Define the DNN model
class DNN(nn.Module):
    def __init__(self, d_in, d_out):  # config.slsum_count, config.dnn_out_d
        super(DNN, self).__init__()
        self.l1 = nn.Linear(d_in, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, d_out)

    def forward(self, x):
        print('x: ', x.cpu().numpy()[0])
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        return out

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, encoding_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, input_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        print(encoded)
        decoded = self.decoder(encoded)
        return decoded
    
class Autoencoder1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Autoencoder1D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(x)
    
class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.fc = nn.Linear(128*4, output_dim)  # Adjust the dimension based on your input size
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.relu(x)


class ConvAutoencoder1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ConvAutoencoder1D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(input_dim, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1)
        )
        self.fc = nn.Linear(1024, output_dim)
    
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.relu(x)

# Instantiate the DNN model
dnn = DNN(8,8)
autoencoder = Autoencoder(8,4)
autoencoder1d = Autoencoder1D(8,8)
conv1d = CNN1D(8,8)
convae = ConvAutoencoder1D(input_dim=1, output_dim=8)

# Pass the input through the DNN model and obtain the output
dnn_output = dnn(x)
autoencoder_output = autoencoder(x)
conv1d_output = conv1d(x.view(1, 1, 8))
convae_output = convae(x.unsqueeze(1))
autoencoder1d_output = autoencoder1d(x)

print('DNN output:', dnn_output.detach().numpy())
print('Autoencoder output:', autoencoder_output.detach().numpy())
print('Autoencoder1D output:', autoencoder1d_output.detach().numpy())
print('CNN1D output:', conv1d_output.detach().numpy())
print('ConvAE output:', convae_output.detach().numpy())

x:  [  0.   8.   1. 243. 234.  17. 244. 206.]
tensor([[150.7112,   0.0000,  47.6473,  70.5854]], grad_fn=<ReluBackward0>)
DNN output: [[ 0.        2.622992 10.750424  0.       16.949675  0.        0.
  23.91987 ]]
Autoencoder output: [[22.039322  0.        4.292387  0.        0.        0.        0.
   0.      ]]
Autoencoder1D output: [[ 0.       40.015133  0.       17.6268    0.        0.        0.
   0.      ]]
CNN1D output: [[ 0.         0.        18.856043  12.74354    7.5623755  0.
  13.958376  27.950138 ]]
ConvAE output: [[ 6.8502784 16.321796   4.4084835  1.3640137  0.         4.06876
   0.         8.598237 ]]
